# Field extraction in the LC gauge

We want to extract the fields necessary to compute $<\pi_i^2>$ in the LC gauge

### Set-up

Set the parameters and environment variables

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 1         
N = 128 
tau_sim = 0.25    
DTS = 8

# Derived parameters
a = L/N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)
nplus = maxt//DTS

# Glasma fields
su_group = 'su3'
uv = 10.0
ir = 0.2
g2mu = 1.5


g = 2.0          		
mu = g2mu / g**2

ns = 50      

nevents = 100

In [2]:
import os

os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.lc_pi_fields as pi_fields

/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using CUDA
Using SU(3)
Using double precision


### Simulation

We define the simulation routine

In [3]:
from tqdm import tqdm

# Simulation rutine
def simulate():
    output = {}
    
    # Derived parameters
    a = L/N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)
    nplus = maxt//DTS
    
    # We create the object simulation
    s = core.Simulation(N, DT, g)

    # We initilize the Glasma fields
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)
    
    # We create objects to store the fields at every time step where the lattices match
    ux_temp = np.zeros((nplus, N*N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
    uy_temp = np.zeros((nplus, N*N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
    Aeta_temp = np.zeros((nplus, N*N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
    
    # We simulate the event and store the fields when the spatial and temporal lattices match
    with tqdm(total=nplus) as pbar:    
        for t in range(maxt):
            
            # Evolve the Glasma fields
            core.evolve_leapfrog(s)
            
            # Store the fields
            if t % DTS == 0:
                
                u1 = s.u1.copy()
                ux_temp[t//DTS] = u1[:, 0, :]
                uy_temp[t//DTS] = u1[:, 1, :]
                
                Aeta_temp[t//DTS] = s.aeta1.copy()
                
                
                pbar.update(1) 
    
    # We free the memory 
    if use_cuda:
        cuda.current_context().deallocations.clear()
        
    # We reshape the fields to use them for the gauge transformation
    ux = ux_temp.reshape(nplus*N*N, su.GROUP_ELEMENTS)
    uy = uy_temp.reshape(nplus*N*N, su.GROUP_ELEMENTS)
    Aeta = Aeta_temp.reshape(nplus*N*N, su.GROUP_ELEMENTS)
    
    # Create the class objects
    pi = pi_fields.GlasmaFields(s, nplus, ux, uy, Aeta)
    
    # We create the objects to store the desired fields
    up = np.zeros((nplus-1, N*N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
    Ay = np.zeros((nplus-1, N*N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
    Az = np.zeros((nplus-1, N*N, su.GROUP_ELEMENTS), dtype=su.GROUP_TYPE)
    
    # We copy the fields to the GPU
    pi.init()
    
    # We gauge transform and extract the fields of the simulation
    with tqdm(total=nplus-1) as pbar:
        for xplus in range(nplus):
            
            # Gauge transform
            pi.compute_fields(xplus, a/hbarc)
            
            # Extract the fields
            up[xplus-1] = pi.up.copy()
            Ay[xplus-1] = pi.ay.copy()
            Az[xplus-1] = pi.az.copy()
            
            pbar.update(1)
    
    # We free the memory 
    if use_cuda:
        cuda.current_context().deallocations.clear()
    
    
    # We write the transformed fields in a dictionary
    output["up"] = up
    output["Ay"] = Ay
    output["Az"] = Az
    
    return output

We perform the simulation

In [4]:
import warnings
warnings.filterwarnings('ignore')

for n in range (nevents):
    
    print('Event %i' % n)
    output = simulate()
    
    up = output['up']
    Ay = output['Ay']
    Az = output['Az']
    
    np.save(os.path.join('simulations', 'pi_lc', 'up_%i.npy' %n), up)
    np.save(os.path.join('simulations', 'pi_lc', 'Ay_%i.npy' %n), Ay)
    np.save(os.path.join('simulations', 'pi_lc', 'Az_%i.npy' %n), Az)

Event 0


100%|██████████| 32/32 [00:03<00:00,  9.96it/s]
32it [00:00, 36.10it/s]                        


Event 1


100%|██████████| 32/32 [00:02<00:00, 15.21it/s]
32it [00:00, 215.36it/s]                        


Event 2


...

100%|██████████| 32/32 [00:02<00:00, 14.62it/s]
32it [00:00, 209.64it/s]                        


Event 98


100%|██████████| 32/32 [00:02<00:00, 14.52it/s]
32it [00:00, 201.37it/s]                        


Event 99


100%|██████████| 32/32 [00:02<00:00, 14.62it/s]
32it [00:00, 208.93it/s]                        
